### Notebook for testing the newest segment anything model by Facebook AI

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFilter
from PIL import ImageOps
import os
from tqdm import tqdm
from sam2.sam2_video_predictor import SAM2VideoPredictor
from sam_interactive_init import show_box, show_mask, show_points, get_frame_names
from util import extract_frames

# See if cuda is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

if device.type == "cuda":
    # Use bfloat16 for faster inference
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

### Extract frames from video

In [ ]:
ROOT_DIR = "../.."
VIDEO_NAME = "back_test"

# Path to jpegs
video_path = os.path.join(ROOT_DIR, VIDEO_NAME, VIDEO_NAME+".mp4")
source_dir = os.path.join(ROOT_DIR, VIDEO_NAME, "frames")       # extracted frames
FPS = 15
extract_frames(video_path, source_dir, FPS, ext="jpg")
pass

### Preview crop

In [ ]:
video_dir = os.path.join(ROOT_DIR, VIDEO_NAME, "crop_frames")   # resized and cropped frames
os.makedirs(video_dir, exist_ok=True)

RESIZE = (144, 312)
# CROP = (0, 110, 144, 280)
CROP = (0, 130, 144, 290)

# Show the first frame
fig, ax = plt.subplots(1, 3, figsize=(5, 5))
frame = Image.open(os.path.join(source_dir, "0001.jpg"))
ax[0].imshow(frame)
frame = frame.resize(RESIZE)
ax[1].imshow(frame)
frame = frame.crop(CROP)
ax[2].imshow(frame)
plt.show()

### Preprocess all frames

In [ ]:
def process_frame(image, crop=CROP, resize=RESIZE):
    # w, h = image.size
    # image = image.resize((w // 2, h // 2))
    image = image.resize(resize)
    image = image.crop(crop)
    # Apply slight gaussian blur
    # image = image.filter(ImageFilter.GaussianBlur(1))
    return image

def reverse_process_frame(image, size, crop=CROP, resize=RESIZE):
    """
    Take a processed frame and reverse the process,
    for example gets 144x170 image, pads to 144x312, then resizes to wxh
    """
    # Zero-pad the image to reverse the crop
    re_w, re_h = resize
    padding = (crop[0], crop[1], re_w - crop[2], re_h - crop[3])
    image = ImageOps.expand(image, padding)
    # Resize the image back to original size
    image = image.resize(size)
    return image

# Crop all frames and save to video_dir
for i, frame in tqdm(enumerate(os.listdir(source_dir)), total=len(os.listdir(source_dir))):
    image = Image.open(os.path.join(source_dir, frame))
    image = process_frame(image)
    image.save(os.path.join(video_dir, frame))

In [ ]:
# scan all the JPEG frame names in this directory
frame_names = get_frame_names(video_dir)

# take a look the first video frame
frame_idx = 0
plt.figure(figsize=(3, 3))
plt.title(f"frame {frame_idx}")
plt.imshow(Image.open(os.path.join(video_dir, frame_names[frame_idx])))

### Init Inference

In [ ]:
# predictor = SAM2VideoPredictor.from_pretrained("facebook/sam2-hiera-large")
# predictor = SAM2VideoPredictor.from_pretrained("facebook/sam2-hiera-base-plus")
# predictor = SAM2VideoPredictor.from_pretrained("facebook/sam2-hiera-small")
predictor = SAM2VideoPredictor.from_pretrained("facebook/sam2-hiera-tiny")
predictor = predictor.to(device)

In [ ]:
inference_state = predictor.init_state(video_path=video_dir,
    offload_video_to_cpu=True,
    offload_state_to_cpu=True,
    async_loading_frames=True,
)

In [ ]:
# Run whenever we want to reset the tracking
predictor.reset_state(inference_state)

### Run Interactive Matplotlib Segmentation Initialization

In [ ]:
# !python sam_interactive_init.py --root_dir "../.." # This doesn't work for the interactive plot
print(os.path.abspath("./sam_interactive_init.py"))
RAN_INTERACTIVE_SAM = True

In [ ]:
if RAN_INTERACTIVE_SAM:
    prompts = torch.load(os.path.join(ROOT_DIR, VIDEO_NAME, "sam_prompts.pth"), weights_only=False)

### Check Init

In [ ]:
if not RAN_INTERACTIVE_SAM:
    ann_frame_idx = 0  # the frame index we interact with
    ann_obj_id = 1  # give a unique id to each object we interact with (it can be any integers)

    # Let's add a positive click at (x, y) to get started
    points = np.array([
        [32, 20],
        [32, 40],
    ], dtype=np.float32)
    # for labels, `1` means positive click and `0` means negative click
    labels = np.array([
        1,
        0,
    ], np.int32)
    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx,
        obj_id=ann_obj_id,
        points=points,
        labels=labels,
    )
else:
    ann_frame_idx = prompts[0]["frame_idx"]
    ann_obj_id = prompts[0]["obj_id"]
    points = prompts[0]["points"]
    labels = prompts[0]["labels"]
    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx,
        obj_id=ann_obj_id,
        points=points,
        labels=labels,
    )

# show the results on the current (interacted) frame
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(Image.open(os.path.join(video_dir, frame_names[ann_frame_idx])))
show_points(points, labels, plt.gca())
show_mask((out_mask_logits[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])

### Propagate

In [ ]:
# run propagation throughout the video and collect the results in a dict
video_segments = {}  # video_segments contains the per-frame segmentation results
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state, 
        start_frame_idx=None,
        max_frame_num_to_track=None,
        reverse=True):
    video_segments[out_frame_idx] = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
    video_segments[out_frame_idx] = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }
    if out_frame_idx > 100:
        break

### Check

In [ ]:
# render the segmentation results every few frames
vis_frame_stride = 15
plt.close("all")
for out_frame_idx in range(0, len(frame_names), vis_frame_stride):
    plt.figure(figsize=(3, 3))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

### Refine

In [ ]:
if False:
    ann_frame_idx = 70  # further refine some details on this frame
    ann_obj_id = 1  # give a unique id to the object we interact with (it can be any integers)

    # show the segment before further refinement
    plt.figure(figsize=(9, 6))
    plt.title(f"frame {ann_frame_idx} -- before refinement")
    plt.imshow(Image.open(os.path.join(video_dir, frame_names[ann_frame_idx])))
    show_mask(video_segments[ann_frame_idx][ann_obj_id], plt.gca(), obj_id=ann_obj_id)

    # Let's add a negative click on this frame at (x, y) to refine the segment
    points = np.array([
        [40, 55],
        [42, 30],
    ], dtype=np.float32)
    # for labels, `1` means positive click and `0` means negative click
    labels = np.array([
        0,
        1,
    ], np.int32)
    _, _, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx,
        obj_id=ann_obj_id,
        points=points,
        labels=labels,
    )

    # show the segment after the further refinement
    plt.figure(figsize=(9, 6))
    plt.title(f"frame {ann_frame_idx} -- after refinement")
    plt.imshow(Image.open(os.path.join(video_dir, frame_names[ann_frame_idx])))
    show_points(points, labels, plt.gca())
    show_mask((out_mask_logits > 0.0).cpu().numpy(), plt.gca(), obj_id=ann_obj_id)

### Mask

In [ ]:
mask_dir = os.path.join(ROOT_DIR, VIDEO_NAME, "mask_frames")
os.makedirs(mask_dir, exist_ok=True)

# Get original size from source_dir
image = Image.open(os.path.join(source_dir, frame_names[0]))
original_size = image.size

# save the segmentation results to disk
for out_frame_idx, out_obj_ids in video_segments.items():
    for out_obj_id, out_mask in out_obj_ids.items():
        mask = (out_mask * 255).astype(np.uint8)
        if mask.ndim == 2:  # Ensure the mask has 3 dimensions (H, W, 1)
            mask = np.expand_dims(mask, axis=-1)
        mask_image = Image.fromarray(mask.squeeze(), mode='L')
        mask_image = reverse_process_frame(mask_image, original_size)
        mask_image.save(os.path.join(mask_dir, f"{out_frame_idx:04d}.png"))

# create a video from the segmentation results
video_save_path = os.path.join(ROOT_DIR, VIDEO_NAME, "mask.mp4")
fps = FPS
os.system(f"ffmpeg -y -r {fps} -f image2 -i {mask_dir}/%04d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p {video_save_path}")

print(os.path.abspath(video_save_path))

### Blur Mask

In [ ]:
mask_blur_dir = os.path.join(ROOT_DIR, VIDEO_NAME, "mask_blur_frames")
os.makedirs(mask_blur_dir, exist_ok=True)

# Blur the mask in the original frames
for out_frame_idx, out_obj_ids in video_segments.items():
    image = Image.open(os.path.join(source_dir, frame_names[out_frame_idx]))
    mask = out_obj_ids[ann_obj_id]
    mask = (mask * 255).astype(np.uint8)
    mask = np.expand_dims(mask, axis=-1)
    mask_image = Image.fromarray(mask.squeeze(), mode='L')
    mask_image = reverse_process_frame(mask_image, original_size)
    mask_image = mask_image.filter(ImageFilter.GaussianBlur(5))
    # Blur the area covered by the mask in the original image
    blur_image = image.copy()
    # blur_image = image.filter(ImageFilter.GaussianBlur(20))

    # Pixelate the area covered by the mask in the original image (do this by resizing the image back and forth)
    pixelate_factor = 50
    blur_image = blur_image.resize((original_size[0] // pixelate_factor, original_size[1] // pixelate_factor))
    blur_image = blur_image.resize(original_size, Image.NEAREST)

    # Paste the mask part of the blurred image on the original image
    image.paste(blur_image, mask=mask_image)
    image.save(os.path.join(mask_blur_dir, f"{out_frame_idx:04d}.jpg"))
    # break

# Plot first frame
plt.figure(figsize=(9, 6))
plt.title(f"frame {0}")
plt.imshow(Image.open(os.path.join(mask_blur_dir, "0000.jpg")))
plt.show()

# create a video from the segmentation results
video_save_path = os.path.join(ROOT_DIR, VIDEO_NAME, "mask_blur.mp4")
fps = FPS
os.system(f"ffmpeg -y -r {fps} -i {mask_blur_dir}/%04d.jpg -vcodec libx264 -crf 25 -pix_fmt yuv420p {video_save_path}")

print(os.path.abspath(video_save_path))